In [1]:
import pandas as pd
import numpy as np 
from numba import jit
import os 

from efficientnet_pytorch import EfficientNet
import codes.mvtecad as mvtec
import codes.battery as battery
from codes.utils import PatchDataset_NCHW, NHWC2NCHW, distribute_scores, distribute_score, save_pickle

/home/kaier/anaconda3/envs/user01/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [24]:
mode = 'train'
normal_class = '코팅부 코팅불량'
fpattern = os.path.join('/home/kaier/KAIER_2022/Battery_data/무지부_코팅부_테이프제외/코팅부', f'{mode}/{normal_class}/*.bmp')

In [14]:
fpattern

'/home/kaier/KAIER_2022/Battery_data/무지부_코팅부/코팅부/train/코팅부 코팅불량/*.bmp'

In [12]:
os.getcwd()

'/home/kaier/KAIER_2022/Member_folder/Yeongkyu/Final_code'

In [25]:
from glob import glob
fpaths = sorted(glob(fpattern))

In [26]:
fpaths

['/home/kaier/KAIER_2022/Battery_data/무지부_코팅부_테이프제외/코팅부/train/코팅부 코팅불량/1_116_2.bmp',
 '/home/kaier/KAIER_2022/Battery_data/무지부_코팅부_테이프제외/코팅부/train/코팅부 코팅불량/1_116_3.bmp',
 '/home/kaier/KAIER_2022/Battery_data/무지부_코팅부_테이프제외/코팅부/train/코팅부 코팅불량/1_12359_1.bmp',
 '/home/kaier/KAIER_2022/Battery_data/무지부_코팅부_테이프제외/코팅부/train/코팅부 코팅불량/1_12376_3.bmp',
 '/home/kaier/KAIER_2022/Battery_data/무지부_코팅부_테이프제외/코팅부/train/코팅부 코팅불량/1_12393_2.bmp',
 '/home/kaier/KAIER_2022/Battery_data/무지부_코팅부_테이프제외/코팅부/train/코팅부 코팅불량/1_12393_3.bmp',
 '/home/kaier/KAIER_2022/Battery_data/무지부_코팅부_테이프제외/코팅부/train/코팅부 코팅불량/1_12_2.bmp',
 '/home/kaier/KAIER_2022/Battery_data/무지부_코팅부_테이프제외/코팅부/train/코팅부 코팅불량/1_150_3.bmp',
 '/home/kaier/KAIER_2022/Battery_data/무지부_코팅부_테이프제외/코팅부/train/코팅부 코팅불량/1_167_2.bmp',
 '/home/kaier/KAIER_2022/Battery_data/무지부_코팅부_테이프제외/코팅부/train/코팅부 코팅불량/1_184_2.bmp',
 '/home/kaier/KAIER_2022/Battery_data/무지부_코팅부_테이프제외/코팅부/train/코팅부 코팅불량/1_184_3.bmp',
 '/home/kaier/KAIER_2022/Battery_data/무지부_코팅부_테이프제외/코팅부/tr

In [13]:
load_path = './save/무지부_코팅부/3_13_efficientnet_b6'
save_path = './save/무지부_코팅부/4_7_efficientnet_b6'

In [3]:
for i in range(1, 7):
    globals()[f'embs_64_tr_{i}'.format(i)] = np.load(f"{load_path}/64_embs_tr_{i}.npy")
    globals()[f'embs_32_tr_{i}'.format(i)] = np.load(f"{load_path}/32_embs_tr_{i}.npy")

In [22]:
@jit
def train_distribution(cov, emb, num_range, I):
    for i in range(num_range):
        cov[i, :, :] = np.cov(emb[:, i, :], rowvar = False) + 0.01 * I    # np.cov(emb[:, i, :]): batch, 344)에 대한 covariance 구함 -> (344,344)
    return cov

@jit
def mahal(u, v, VI):
    VI = np.atleast_2d(VI)
    delta = u - v
    p1 = np.dot(delta, VI)
    m = np.dot(p1, delta)
    return np.sqrt(m)


@jit
def test_inference(mean, cov, num_range, emb):

    dist_list = []

    for i in range(num_range):
        partial_mean = mean[i, :]
        conv_inv = np.linalg.inv(cov[i, :, :])
        dist = [mahal(sample[i, :], partial_mean, conv_inv) for sample in emb]
        dist_list.append(dist)

    return dist_list

In [14]:
def infer(embs_tr, embs_te, K, S):
        
    B_train, H_train, W_train, C_train = embs_tr.shape   # embs_tr: [batch, 9, 9, 344]
    emb_train = embs_tr.reshape((B_train, H_train * W_train, C_train))  

    del embs_tr

    mean_train = np.mean(emb_train, axis = 0)   # mean_train: (81, 344)
    cov_train = np.zeros((H_train * W_train, C_train, C_train))   # cov_train: (81, 344, 344)

    I = np.identity(C_train) # I: (344, 344)

    # covariance 계산을 numba로 처리.
    cov_train = train_distribution(cov_train, emb_train, H_train*W_train, I)
    

    del emb_train  
    del I

#     mem_report("After Train distribution") # memory usage checking.

    B_test, H_test, W_test, C_test = embs_te.shape 
    emb_test = embs_te.reshape((B_test, H_test * W_test, C_test))   # (batch, 9*9, 344)

    del embs_te

    #dist_list = []

    cov_train = cov_train.astype('float32')

    
#     train_dist_list = test_inference
    # Test inference를 대체
    dist_list = test_inference(mean_train, cov_train, H_test*W_test, emb_test)

    dist_list = np.array(dist_list).transpose(1, 0).reshape(B_test, H_test, W_test)   # (batch, 9, 9)
    
    return dist_list

In [15]:
def assess_anomaly_maps(anomaly_maps, category):
    print("Anomaly Map shape: ", anomaly_maps.shape)
    
#     anomaly_kl_scores = np.zeros(anomaly_maps.shape[0])
    
#     for i in range(anomaly_maps.shape[0]):
#         anomaly_kl_scores[i] = scipy.stats.entropy(normal_map.flatten(), anomaly_maps[i].flatten())
     
    Det_AUROC = dict()
    anomaly_scores = anomaly_maps.max(axis=-1).max(axis=-1)
    np.save(f"{save_path}/train_{category}_anomaly_scores", anomaly_scores)
    final_det_auroc = 0 
    
    return final_det_auroc

In [18]:
def inference_procedure(x_tr_64, x_tr_32, x_te_64, x_te_32,  idx_=1):
    
    maps_64 = infer(x_tr_64, x_te_64, K=64, S=24)
    maps_64 = distribute_scores(maps_64, (256, 256), K=64, S=24)
    #det_64 = assess_anomaly_maps(maps_64, "det_64")
    maps_32 = infer(x_tr_32, x_te_32, K=32, S=14)
    maps_32 = distribute_scores(maps_32, (256, 256), K=32, S=14) 
    #det_32 = assess_anomaly_maps(maps_32, "det_32")

    maps_mult = maps_64 * maps_32
    
    det_mult = assess_anomaly_maps(maps_mult, f"model_{idx_}")
    
    return det_mult

In [4]:
embs_64_tr_1.shape

(9508, 9, 9, 344)

In [8]:
embs_64_te = np.concatenate((embs_64_tr_1, embs_64_tr_2, embs_64_tr_3, embs_64_tr_4, embs_64_tr_5, embs_64_tr_6), axis=0)
embs_64_te.shape

(12400, 9, 9, 344)

In [9]:
embs_32_te = np.concatenate((embs_32_tr_1, embs_32_tr_2, embs_32_tr_3, embs_32_tr_4, embs_32_tr_5, embs_32_tr_6), axis=0)
embs_32_te.shape

(12400, 17, 17, 344)

In [23]:
tmp = inference_procedure(embs_64_tr_1, embs_32_tr_1, embs_64_te, embs_32_te, idx_=1)

Anomaly Map shape:  (12400, 256, 256)


In [24]:
tmp = inference_procedure(embs_64_tr_2, embs_32_tr_2, embs_64_te, embs_32_te, idx_=2)

Anomaly Map shape:  (12400, 256, 256)


In [25]:
tmp = inference_procedure(embs_64_tr_3, embs_32_tr_3, embs_64_te, embs_32_te, idx_=3)

Anomaly Map shape:  (12400, 256, 256)


In [26]:
tmp = inference_procedure(embs_64_tr_4, embs_32_tr_4, embs_64_te, embs_32_te, idx_=4)

Anomaly Map shape:  (12400, 256, 256)


In [27]:
tmp = inference_procedure(embs_64_tr_5, embs_32_tr_5, embs_64_te, embs_32_te, idx_=5)

Anomaly Map shape:  (12400, 256, 256)


In [28]:
tmp = inference_procedure(embs_64_tr_6, embs_32_tr_6, embs_64_te, embs_32_te, idx_=6)

Anomaly Map shape:  (12400, 256, 256)
